In [4]:
import whisper
import torch

# First, define the model size
MODEL_SIZE = "medium"  # or "tiny", "base", "small", "large"

def comprehensive_model_analysis(model_size="medium"):
    """Comprehensive analysis of Whisper model"""
    model = whisper.load_model(model_size)
    
    print(f"=== Whisper {model_size.upper()} Model Analysis ===")
    
    # Model structure
    print(f"\nModel structure:")
    for name, module in model.named_children():
        num_params = sum(p.numel() for p in module.parameters())
        print(f"  {name}: {type(module).__name__} - {num_params:,} parameters")
    
    # Encoder details
    if hasattr(model, 'encoder'):
        encoder = model.encoder
        print(f"\nEncoder details:")
        print(f"  Input dimensions: {getattr(encoder, 'd_model', 'N/A')}")
        if hasattr(encoder, 'blocks'):
            print(f"  Number of blocks: {len(encoder.blocks)}")
            # Analyze first block
            if len(encoder.blocks) > 0:
                first_block = encoder.blocks[0]
                print(f"  First block structure:")
                for name, module in first_block.named_children():
                    params = sum(p.numel() for p in module.parameters())
                    print(f"    {name}: {params:,} params")
    
    # Decoder details  
    if hasattr(model, 'decoder'):
        decoder = model.decoder
        print(f"\nDecoder details:")
        print(f"  Output dimensions: {getattr(decoder, 'd_model', 'N/A')}")
        if hasattr(decoder, 'blocks'):
            print(f"  Number of blocks: {len(decoder.blocks)}")
    
    # Memory requirements
    param_count = sum(p.numel() for p in model.parameters())
    param_size = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024**2)
    print(f"\nMemory requirements:")
    print(f"  Total parameters: {param_count:,}")
    print(f"  Model size: {param_size:.2f} MB")
    print(f"  Estimated inference memory: {param_size * 4:.2f} MB (rough estimate)")

# Run the analysis
comprehensive_model_analysis(MODEL_SIZE)

=== Whisper MEDIUM Model Analysis ===

Model structure:
  encoder: AudioEncoder - 305,680,384 parameters
  decoder: TextDecoder - 456,641,536 parameters

Encoder details:
  Input dimensions: N/A
  Number of blocks: 24
  First block structure:
    attn: 4,197,376 params
    attn_ln: 2,048 params
    mlp: 8,393,728 params
    mlp_ln: 2,048 params

Decoder details:
  Output dimensions: N/A
  Number of blocks: 24

Memory requirements:
  Total parameters: 762,321,920
  Model size: 2908.03 MB
  Estimated inference memory: 11632.11 MB (rough estimate)


In [5]:
import whisper
import torch

# Define model size
MODEL_SIZE = "medium"  # Change to "tiny", "base", "small", "large" as needed

def analyze_whisper_layers(model_size=MODEL_SIZE):
    """Complete analysis of Whisper model layers and structure"""
    
    print(f"Loading Whisper {model_size} model...")
    model = whisper.load_model(model_size)
    print("✅ Model loaded!\n")
    
    # 1. Basic model structure
    print("=== BASIC MODEL STRUCTURE ===")
    for name, module in model.named_children():
        num_params = sum(p.numel() for p in module.parameters())
        print(f"{name:15} | {type(module).__name__:20} | {num_params:>12,} parameters")
    
    # 2. Encoder analysis
    print("\n=== ENCODER ANALYSIS ===")
    if hasattr(model, 'encoder'):
        encoder = model.encoder
        print(f"Encoder type: {type(encoder).__name__}")
        
        if hasattr(encoder, 'blocks'):
            print(f"Number of encoder blocks: {len(encoder.blocks)}")
            
            # Analyze individual encoder blocks
            total_encoder_params = 0
            for i, block in enumerate(encoder.blocks):
                block_params = sum(p.numel() for p in block.parameters())
                total_encoder_params += block_params
                block_size_mb = sum(p.numel() * p.element_size() for p in block.parameters()) / (1024 * 1024)
                print(f"  Encoder Block {i:2d}: {block_params:>10,} params, {block_size_mb:>6.2f} MB")
            
            print(f"Total encoder parameters: {total_encoder_params:,}")
        
        # Encoder configuration
        if hasattr(encoder, 'd_model'):
            print(f"Model dimension (d_model): {encoder.d_model}")
        if hasattr(encoder, 'num_heads'):
            print(f"Number of attention heads: {encoder.num_heads}")
    
    # 3. Decoder analysis
    print("\n=== DECODER ANALYSIS ===")
    if hasattr(model, 'decoder'):
        decoder = model.decoder
        print(f"Decoder type: {type(decoder).__name__}")
        
        if hasattr(decoder, 'blocks'):
            print(f"Number of decoder blocks: {len(decoder.blocks)}")
            
            # Analyze individual decoder blocks
            total_decoder_params = 0
            for i, block in enumerate(decoder.blocks):
                block_params = sum(p.numel() for p in block.parameters())
                total_decoder_params += block_params
                block_size_mb = sum(p.numel() * p.element_size() for p in block.parameters()) / (1024 * 1024)
                print(f"  Decoder Block {i:2d}: {block_params:>10,} params, {block_size_mb:>6.2f} MB")
            
            print(f"Total decoder parameters: {total_decoder_params:,}")
    
    # 4. Memory requirements
    print("\n=== MEMORY REQUIREMENTS ===")
    total_params = sum(p.numel() for p in model.parameters())
    model_size_mb = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 * 1024)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Model size (storage): {model_size_mb:.2f} MB")
    print(f"Estimated inference RAM: {model_size_mb * 4:.2f} MB")
    print(f"Estimated VRAM for GPU: {model_size_mb * 1.5:.2f} MB")
    
    # 5. Layer distribution suggestions
    print("\n=== DISTRIBUTION SUGGESTIONS ===")
    if hasattr(model, 'encoder') and hasattr(model.encoder, 'blocks'):
        encoder_blocks = len(model.encoder.blocks)
        print(f"Encoder has {encoder_blocks} blocks - ideal for splitting across machines")
        
        # Suggest splits
        for num_splits in [2, 4, 8]:
            if encoder_blocks >= num_splits:
                blocks_per_split = encoder_blocks // num_splits
                remainder = encoder_blocks % num_splits
                print(f"  {num_splits}-way split: ~{blocks_per_split} blocks per machine" + 
                      (f" (+{remainder} extra)" if remainder else ""))
    
    return model

# Run the complete analysis
model = analyze_whisper_layers(MODEL_SIZE)

Loading Whisper medium model...
✅ Model loaded!

=== BASIC MODEL STRUCTURE ===
encoder         | AudioEncoder         |  305,680,384 parameters
decoder         | TextDecoder          |  456,641,536 parameters

=== ENCODER ANALYSIS ===
Encoder type: AudioEncoder
Number of encoder blocks: 24
  Encoder Block  0: 12,595,200 params,  48.05 MB
  Encoder Block  1: 12,595,200 params,  48.05 MB
  Encoder Block  2: 12,595,200 params,  48.05 MB
  Encoder Block  3: 12,595,200 params,  48.05 MB
  Encoder Block  4: 12,595,200 params,  48.05 MB
  Encoder Block  5: 12,595,200 params,  48.05 MB
  Encoder Block  6: 12,595,200 params,  48.05 MB
  Encoder Block  7: 12,595,200 params,  48.05 MB
  Encoder Block  8: 12,595,200 params,  48.05 MB
  Encoder Block  9: 12,595,200 params,  48.05 MB
  Encoder Block 10: 12,595,200 params,  48.05 MB
  Encoder Block 11: 12,595,200 params,  48.05 MB
  Encoder Block 12: 12,595,200 params,  48.05 MB
  Encoder Block 13: 12,595,200 params,  48.05 MB
  Encoder Block 14: 12,

In [ ]:
def plan_distributed_setup(model_size="medium", num_machines=2):
    """Plan how to distribute Whisper across multiple machines"""
    
    model = whisper.load_model(model_size)
    
    print(f"=== DISTRIBUTION PLAN for Whisper {model_size.upper()} across {num_machines} machines ===\n")
    
    if hasattr(model, 'encoder') and hasattr(model.encoder, 'blocks'):
        encoder_blocks = model.encoder.blocks
        total_blocks = len(encoder_blocks)
        
        blocks_per_machine = total_blocks // num_machines
        remainder = total_blocks % num_machines
        
        print(f"Total encoder blocks: {total_blocks}")
        print(f"Target: {num_machines} machines")
        print(f"Base blocks per machine: {blocks_per_machine}")
        if remainder:
            print(f"Extra blocks to distribute: {remainder}")
        
        print("\nRecommended distribution:")
        current_block = 0
        for machine in range(num_machines):
            machine_blocks = blocks_per_machine
            if machine < remainder:
                machine_blocks += 1
            
            block_range = f"{current_block}-{current_block + machine_blocks - 1}"
            
            # Calculate parameters for this machine's blocks
            machine_params = 0
            for i in range(current_block, current_block + machine_blocks):
                machine_params += sum(p.numel() for p in encoder_blocks[i].parameters())
            
            machine_size_mb = machine_params * 4 / (1024 * 1024)  # approx size in MB
            
            print(f"  Machine {machine}: Blocks {block_range} | {machine_params:,} params | ~{machine_size_mb:.1f} MB")
            current_block += machine_blocks
    
    return model

# Test the distribution planning
plan_distributed_setup(MODEL_SIZE, num_machines=2)

NameError: name 'MODEL_SIZE' is not defined